In [1]:
import tensorflow as tf
from keras.models import load_model
from keras.layers import *
from keras import Model, Input
from keras import optimizers
import keras.backend as K
import numpy as np
import pickle


/home/henrik/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def get_darknet():
    model = load_model('obj/darknet_mod.h5')
    for layer in model.layers:
        layer.trainable = False
    return model

def make_content_model():
    darknet = get_darknet()
    
    # pretrained layers
    inputs = darknet.layers[0].input
    x = darknet.get_layer('conv2d_4').output
    base_model = Model([inputs], [x])
    input_content = Input((256,256,3))
    input_output = Input((256,256,3))
    
    content_features = base_model(input_content)
    output_features = base_model(input_output)
    
    loss = Lambda(contentLoss, output_shape=(1,))([content_features, output_features])
    
    return Model([input_content, input_output], [loss])


def contentLoss(x):
    n1, n2, n3 = map(int, x[0].shape[1:])
    return K.flatten(K.sum(K.square(x[0] - x[1]), keepdims=True))

In [3]:
content_model = make_content_model()
content_model.save('obj/content_model.h5')

/home/henrik/miniconda3/lib/python3.6/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
